## Obtencion del dataset

### Paso 1: Instalar Dependencias para el proyecto

In [4]:
%pip install roboflow tensorflow

Note: you may need to restart the kernel to use updated packages.


### Paso 2: Descargar el dataset de Roboflow

In [5]:

from roboflow import Roboflow
rf = Roboflow(api_key="nK6spHh5hG9qjWGCbZ43")
project = rf.workspace("miguellopez").project("image-classification-jtsve")
version = project.version(1)
dataset = version.download("folder")
                
                

loading Roboflow workspace...
loading Roboflow project...


### Paso 3: Cargar dataset con ImageDataGenerator

In [6]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224,224)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% para entrenamiento, 20% para validación
)

train_generator = datagen.flow_from_directory(
    dataset.location + "/train",
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Conjunto de entrenamiento
)

validation_generator = datagen.flow_from_directory(
    dataset.location + "/train",
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Conjunto de validación
)


Found 1646 images belonging to 7 classes.
Found 409 images belonging to 7 classes.


### Paso 4: Realizar el preprocesamiento del dataset

 En este caso se estan transformando las imagenes y posteriormente guardando en la carpeta "preprocesado" para poder verificar los cambios realizados durante el proceso. Sin embargo, usar ImageDataGenerator es más útil porque aplica las transformaciones en tiempo real durante el entrenamiento de un modelo, esto debido a que:

  - Ahorra espacio
  - Es más rápido
  - Mejora la generalización del modelo
  - Permite ajustar fácilmente los aumentos.

In [10]:
import os
import cv2
import numpy as np
import random

# Rutas
# tomamos como directorio de entrada el directorio de entrenamiento del dataset
input_dir = dataset.location + "/train"
output_dir = "preprocesado"

# Rango de transformaciones para cada paraámetro 
rotation_range = [-15, 15]      # grados (mín, máx)
scale_range = [0.8, 1.2]        # factores de escala (mín, máx)
brightness_range = [0.3, 1.8]   # factor de brillo (mín, máx)

num_augmentations = 1           # maximo de transformaciones por imagen

# Crear carpeta de salida
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Procesar cada emoción
for emotion in os.listdir(input_dir):
    emotion_path = os.path.join(input_dir, emotion)
    save_path = os.path.join(output_dir, emotion)
    os.makedirs(save_path, exist_ok=True)

    for filename in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, filename)
        img = cv2.imread(img_path)

        if img is None:
            continue

        # Guardar imagen original
        # cv2.imwrite(os.path.join(save_path, f"original_{filename}"), img)

        # Generar N transformaciones aleatorias por imagen
        for i in range(num_augmentations):

            # 1. Brillo
            b = random.uniform(brightness_range[0], brightness_range[1])
            bright_img = cv2.convertScaleAbs(img, alpha=b, beta=0)
            
            # 2. Rotación
            angle = random.uniform(rotation_range[0], rotation_range[1])
            h, w = img.shape[:2]
            center = (w // 2, h // 2)
            matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated_img = cv2.warpAffine(bright_img, matrix, (w, h))
            
            # 3. Escalado
            s = random.uniform(scale_range[0], scale_range[1])
            new_size = (int(w * s), int(h * s))
            resized_img = cv2.resize(rotated_img, new_size)
            
            # Guardar la imagen transformada
            cv2.imwrite(
                os.path.join(save_path, f"aug_{i}_b{b:.1f}_r{angle:.1f}_s{s:.1f}_{filename}"),
                resized_img
            )